In [1]:
import sys
import os

In [4]:
import torch

ImportError: libcudart.so.10.0: cannot open shared object file: No such file or directory

In [2]:
sys.path.append("..")
from train.parser import argument_parser

ImportError: libcudart.so.10.0: cannot open shared object file: No such file or directory

In [ ]:
root = os.path.abspath('..')
args = argument_parser()
model_dir = os.path.join(root, args.model_dir)
# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')